## Intro to SQLTools - DBAccess
- A DSA python package to facilitate interaction with relational databases from python
- Gitlab repo
    - lunar-sql (lunar house staging)
      http://gitlab.hodac.phz:8000/data-science/lunar-sql/blob/development/src/ho/sqltools/dbaccess.py
    - sqltools (rhel / acp)
- Available from Artifactory

In [ ]:
# To install the package
#
# conda install ho_sqltools

In [ ]:
# import the package
from ho.sqltools import DBAccess
import pandas as pd

In [ ]:
# Default parameters

# DBAccess(host=None, database=None, user=None, password=None,
#          drivername='postgresql+psycopg2', port=None, reuse_connection=False)


In [ ]:
# Database connection parameters (example is Redshift MO)
# if user and / or password not specified this is taken from pgpass 

host='ma-red-andb-m01.cfbhyo4wotof.eu-west-2.redshift.amazonaws.com'
database='aws_red_dacc'
user='crowler'
# password = 'your_password'
drivername = 'redshift+pyscopg2'
port='5432'

In [ ]:
# create connection
dba = DBAccess(host=host, database=database, user=user, drivername=drivername, reuse_connection=True)

In [ ]:
dba

In [ ]:
# load a table
schema = 'sandpit_crowler'
table = 'metrics_demo'

In [ ]:
metrics = dba.load_table(table=table, schema=schema, columns='*', limit=10, order_by='metric_datetime', to='df')

In [ ]:
metrics.head()

In [ ]:
metrics.info()

In [ ]:
# load_select
sql = f"""select *
          from {schema}.{table}
          where reporting_count_1 > 5450"""
select_metrics = dba.load_select(sql)
select_metrics

In [ ]:
# write table
write_table_name = 'metrics_demo_write'
dba.write_table(select_metrics, write_table_name, schema, index=False, if_exists='append')

In [ ]:
# grant on table
dba.grant_on_table(write_table_name, schema, to='crowler')

In [ ]:
# check schema exists
dba.schema_exists(schema)

In [ ]:
# check table exists
dba.check_table_exists_in_schema(write_table_name, schema)

In [ ]:
dba.get_table_names_in_schema(schema)[:5]

In [ ]:
# get column names
dba.get_column_names_in_table(write_table_name, schema, sort_alphabetically=True)

In [ ]:
# copy tables (to another schema), assumes you have permisions
# your_sandpit = ''
# dba.copy_tables(from_sandpit='sandpit_crowler', to_sandpit=your_sandpit, tables=write_table_name,
#                replace=False)
# dba.check_table_exists_in_schema(write_table_name, your_sandpit)

In [ ]:
# drop table
dba.drop_table(write_table_name, schema)
dba.check_table_exists_in_schema(write_table_name, schema)

In [ ]:
dba.count_num_rows(table, schema)

In [ ]:
# Primary connection - use to execute any sql
sql = f"""
insert into {schema}.{table} (metric_datetime, reporting_count_1, reporting_count_2)
values ('2022-12-02 12:00:00', 9999, 9999)"""
dba.primary_connection.execute(sql)
dba.count_num_rows(table, schema)

In [ ]:
# Use with sqlalchemy
from sqlalchemy import Table, MetaData, insert

new_values = {'metric_datetime': '2022-12-02 12:00:00',
             'reporting_count_1': 9999,
             'reporting_count_2': 9999}

metadata = MetaData(bind=dba.engine, schema=schema)
sqlalchemy_table = Table(table, metadata, autoload_with=dba.engine)

insert_statement = insert(sqlalchemy_table).values(**new_values)

dba.primary_connection.execute(insert_statement)

dba.count_num_rows(table, schema)

In [ ]:
# Use with Pandas - Read
sql = f"""
select metric_datetime, reporting_count_1, reporting_count_2
from {schema}.{table}
order by metric_datetime desc
limit 1"""
df = pd.read_sql(sql, dba.primary_connection)
df

In [ ]:
# Use with Pandas - Write
df.to_sql(table, 
          dba.primary_connection,
          schema=schema,
          method='multi',
          if_exists='append',
          index=False)
dba.count_num_rows(table, schema)

In [ ]:
# close connection
dba.close()